In [2]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 710.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 100.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install torchdata==0.5.1 torchtext==0.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.6.1
    Uninstalling torchdata-0.6.1:
      Successfully uninstalled torchdata-0.6.1


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu117.html

Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.0 MB/s eta 0:00:00


In [5]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=58bafe20a9bc80bdd4f034f94dd146a9c637000868f8a960f858a1954e1f765f
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [6]:
from torch_geometric.nn import GCNConv
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,GATv2Conv
from torch_scatter import scatter_mean
from torch_geometric.data import InMemoryDataset, download_url, extract_zip
from torch_geometric.nn import MetaLayer

In [7]:
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu" )
device

device(type='cuda')

In [8]:
def get_int_map(dep):
    dep = df.loc[df["deployment"]==dep]
    dep = dep.reset_index(drop=True)
    return dep

In [11]:
df = pd.read_csv("combined_24sce_with_num_AP_STA.csv")

In [12]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', 'wlan_code_index', 'x(m)', 'y(m)', 'z(m)',
       'primary_channel', 'min_channel_allowed', 'max_channel_allowed',
       'node_type', 'rssi', 'sinr', 'air_time_mean', 'channel_bonding_model',
       'airtime_0', 'airtime_1', 'airtime_2', 'airtime_3', 'airtime_4',
       'airtime_5', 'airtime_6', 'airtime_7', 'deployment', 'throughput',
       'central_freq', 'num_AP', 'num_STA'],
      dtype='object')

In [13]:
df.drop(["Unnamed: 0", "Unnamed: 0.1"],axis=1,inplace=True)

In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7,deployment,throughput,central_freq,num_AP,num_STA
0,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,-103.64,...,50.3,0.0,0.0,0.0,0.0,0.0,104.96,5.0,12,10
1,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,-103.64,...,50.3,0.0,0.0,0.0,0.0,0.0,7.68,5.0,12,10
2,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,-103.64,...,50.3,0.0,0.0,0.0,0.0,0.0,11.09,5.0,12,10
3,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,-103.64,...,50.3,0.0,0.0,0.0,0.0,0.0,14.51,5.0,12,10
4,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,-103.64,...,50.3,0.0,0.0,0.0,0.0,0.0,11.95,5.0,12,10


In [15]:
df.shape[0]

382840

In [16]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# test_1 = df[['0', '1', '2', '3', '4', '5', '6', '7','8', '9', '10', '11', 'wlan_code_index', 'x(m)', 'y(m)','z(m)',
#             'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'node_type','rssi',
#             'sinr', 'air_time_mean','channel_bonding_model', "airtime_0", "airtime_1", "airtime_2",
#               "airtime_3", "airtime_4", "airtime_5", "airtime_6", "airtime_7","central_freq", 'deployment']]
# print(test_1.columns)
# test_1.drop(["deployment"],axis=1,inplace=True)
# print(test_1.columns)

In [17]:
# Creating individual graphs
# This assumes all APs and STAs are connected to each other
def create_graph(split,split_y,deployment):
    dep = get_int_map(deployment)
    dep_y = dep["throughput"]
    dep_x = dep[['0', '1', '2', '3', '4', '5', '6', '7','8', '9', '10', '11','deployment', 'wlan_code_index', 'x(m)', 'y(m)','z(m)',
            'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'node_type','rssi',
            'sinr', 'air_time_mean','channel_bonding_model',"airtime_0", "airtime_1", "airtime_2",
              "airtime_3", "airtime_4", "airtime_5", "airtime_6", "airtime_7"]]
    #print(dep_x)
    dep_reset = dep.reset_index(drop=True)
    ap_index = {}
    out = dep_reset[dep_reset["node_type"] == 0]
    for i in range(len(out)):
        ap_index[out.index[i]] = i
    #print(ap_index)
    node_features = dep_x.iloc[:,13:].values
    # print(node_features)
    #edge_features = dep.iloc[:,:12].values - here each node has been given an edge feature
    # need to give each edge an edge feature
    node_targets = dep_y.values
    node_features = torch.tensor(node_features, dtype=torch.float)
    print(node_features.shape)
    #edge_features = torch.tensor(edge_features, dtype=torch.float)
    node_targets = torch.tensor(node_targets, dtype=torch.float)
    # Add edges here for each deployment
    edges = []
    edge_features = []
    edge_index = []
    for i in range(len(dep)):
        for j in range(len(dep)):
            if (i != j and (dep["node_type"].iloc[i] == 0 and dep["node_type"].iloc[j] == 0)) or (i !=j and (dep["node_type"].iloc[i] == 1 and dep["node_type"].iloc[j] == 0)):
                edges.append([i,j])
    #print(edges)
    edges2=edges
    edges = torch.tensor(edges, dtype=torch.float)
    #print("Edges: ", edges, edges.shape)
    # edge_index = torch.tensor(edges, dtype=torch.long)
    edge_index = torch.tensor(edges,dtype=torch.long)
    edge_index = edge_index.t().contiguous()
    #print(edges.detach(), edges.shape)


    #tpml station interference
    # ap_dist=0
    # distance=0

    # for j in range(len(out)):
    #     for i,val in dep.iterrows():
    #         i_pos = np.asarray([dep.at[out.index[j],"x(m)"],dep.at[out.index[j],"y(m)"],dep.at[out.index[j],"z(m)"]])
    #         j_pos = np.asarray([dep.at[i,"x(m)"],dep.at[i,"y(m)"],dep.at[i,"z(m)"]])
    #         distance = np.linalg.norm(i_pos - j_pos)
    #         if val["node_type"]==0:
    #             ap_dist=distance
    #         else:
    #             if ap_dist!=0:
    #                 new_int=val[str(j)] + 10*np.log10((ap_dist/distance)**2)
    #                 dep.at[i, str(j)]=new_int

    print(edges.shape[0])
    for i in range(edges.shape[0]):
        # print(dep.iloc[int(edges[i][0]), ap_index[int(edges[i][1])]])
        i_pos = np.asarray([dep.at[edges2[i][0],"x(m)"],dep.at[edges2[i][0],"y(m)"],dep.at[edges2[i][0],"z(m)"]])
        j_pos = np.asarray([dep.at[edges2[i][1],"x(m)"],dep.at[edges2[i][1],"y(m)"],dep.at[edges2[i][1],"z(m)"]])
        distance = np.linalg.norm(i_pos - j_pos)
        #print(i)
        # edge_features.append([distance,dep.iloc[int(edges[i][0]), ap_index[int(edges[i][1])]]])
        edge_type = 0
        if int(edges2[i][0]) in ap_index.keys():
          edge_type = 0
        else:
          edge_type = 1
        edge_features.append([edge_type,distance,dep.at[edges2[i][0],"rssi"],dep.iloc[int(edges[i][0]), ap_index[int(edges[i][1])]]])
    # edge_features = np.array(edge_features)
    # print(edge_features)
    edge_features = torch.tensor(edge_features, dtype=torch.float)
    #print(edge_features, edge_features.shape)
    graph = {
        "edges": edges,
        "edge_index": edge_index,
        "node_features": node_features,
        "edge_features": edge_features,
        "node_targets": node_targets
    }
#     print("*"*10)
#     print(edges.shape)
#     print(edge_index.shape)
#     print(node_features.shape)
#     print(edge_features.shape)
#     print(node_targets.shape)
    return graph

In [ ]:
create_graph(0,0,0)

In [ ]:
create_graph(0,0,0)["node_features"][0]


In [20]:
def create_geometric_graph(graph):
    data = Data(
        # Input graph.
        x=graph["node_features"],
        #pos=pos,
        edge_index=graph["edge_index"],
        edge_attr=graph["edge_features"],
        # Output node targets.
        y=graph["node_targets"],
        num_nodes = len(graph["node_features"])

    )

    return data

In [ ]:
print(create_geometric_graph(create_graph(0,0,0)))

In [ ]:

2400-1920

480

In [22]:
# With train, validation and test data.
import random
import os
from torch_geometric.data import InMemoryDataset, download_url, extract_zip
# divide into training and testing points
class CustomDataset(InMemoryDataset):
    def __init__(self, split="train", transform=None):
        self.data = pd.read_csv("combined_24sce_with_num_AP_STA.csv")
        self.split = split
        super(CustomDataset, self).__init__( split, transform)
        #self.data, self.slices = torch.load(self.processed_paths[0])
        #self.data = pd.read_csv("deployment_with_int_map.csv")
        #self.data, self.slices = pd.read_csv("deployment_with_int_map.csv")

        # print("In init")

    @property
    def raw_file_names(self):
        # print("In raw_file_names")
        return ["combined_24sce_with_num_AP_STA.csv"]

    @property
    def processed_file_names(self):
        # print("In processed_file_names")
        li = ['data_train_' + str(i) + '.pt' for i in range(1440)]+ ['data_valid_' + str(j) + '.pt' for j in range(1440, 1920)] + ['data_test_' + str(k) + '.pt' for k in range(1920, 2400)]
        #print(li)
        return ['data_train_' + str(i) + '.pt' for i in range(1440)]+ ['data_valid_' + str(j) + '.pt' for j in range(1440,1920)] + ['data_test_' + str(k) + '.pt' for k in range(1920,2400)]

    def _download(self):
        '''
        print("In download")
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        # The zip file is removed
        os.unlink(path)
        '''
        pass

    def process(self):
        print("In process")
        #df = pd.read_csv(self.raw_paths[0])
        X = self.data[['0', '1', '2', '3', '4', '5', '6', '7','8', '9', '10', '11', 'wlan_code_index', 'x(m)', 'y(m)','z(m)',
            'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'node_type','rssi',
            'sinr', 'air_time_mean','channel_bonding_model', "airtime_0", "airtime_1", "airtime_2",
              "airtime_3", "airtime_4", "airtime_5", "airtime_6", "airtime_7"]]
        y = self.data.loc[:, ["throughput", "deployment"]]
#         X_train = X.iloc[:183854, :]
#         X_valid = X.iloc[183854:245139, :]
#         X_test = X.iloc[245139:,:]
#         print(X_test.columns)
#         y_train = y.iloc[:183854, :]
#         y_valid = y.iloc[183854:245139, :]
#         y_test = y.iloc[245139:,:]
        graphs = []
        self.l_train=[]
        self.l_valid=[]
        self.l_test=[]

        # print("Here")
        for i in range(3, 9):
            sce = [i for i in range(320 * (i - 3), 320 * (i - 2))] + [i for i in range(1920 + 80 * (i - 3), 1920 + 80 * (i - 2))]
            self.l_train.extend(random.sample(sce, 240))
            sce = [x for x in sce if x not in self.l_train]
            self.l_valid.extend(random.sample(sce, 80))
            sce = [x for x in sce if x not in self.l_valid]
            self.l_test.extend(sce)
        # l = [i for i in range(1920)]
        # self.l_train = random.sample(l, 1440)
        # # l = [x for x in l if x not in self.l_train]
        # self.l_valid = [1,2,3]
        # # l = [x for x in l if x not in self.l_valid]
        # self.l_test =random.sample([i for i in range(1920,2400)], 480)
        count = 0
        if(self.split == "train"):

            for i in self.l_train:

                #X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float)
                #y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float)
                graph = create_graph(X, y, i)

                graph = create_geometric_graph(graph)
                graphs.append(graph)

                torch.save(graph, os.path.join(self.processed_dir, f'data_train_{count}.pt'))
                count += 1
        elif(self.split == "valid"):
            for i in self.l_valid:
                #X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float)
                #y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float)
                graph = create_graph(X, y, i)
                graph = create_geometric_graph(graph)
                graphs.append(graph)

                torch.save(graph, os.path.join(self.processed_dir, f'data_valid_{count}.pt'))
                count += 1
        else:
            for i in self.l_test:
                #X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float)
                #y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float)
                graph = create_graph(X, y, i)
                graph = create_geometric_graph(graph)
                graphs.append(graph)

                torch.save(graph, os.path.join(self.processed_dir, f'data_test_{count}.pt'))
                count += 1
        #return graphs[idx]

    def __len__(self):
        if(self.split == "train"):
            #return len(self.processed_file_names[0])
            return 1440
        elif self.split == "valid":
            return 480
        else:
            return 480

    def __getitem__(self, idx):
        #print("Part: ", self.processed_file_names[1])

        if(self.split == "train"):
            data = torch.load(os.path.join(self.processed_dir, f'data_train_{idx}.pt'))
        elif(self.split == "valid"):
            data = torch.load(os.path.join(self.processed_dir, f'data_valid_{idx}.pt'))
        elif (self.split=="test"):
            data = torch.load(os.path.join(self.processed_dir, f'data_test_{idx}.pt'))
        return data

In [ ]:
dataset_train = CustomDataset( split='train')
dataset_valid = CustomDataset( split='valid')
dataset_test = CustomDataset( split='test')

# MODS TO MODEL START HERE

In [87]:
#TPGAT

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

class EdgeModel(torch.nn.Module):
    def __init__(self, n_node_features, n_edge_features, hiddens, n_targets):
        super().__init__()
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(2 * n_node_features + n_edge_features, hiddens),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hiddens, n_targets),
        )

    def forward(self, src, dest, edge_attr, u=None, batch=None):
        #print("In edge model")
        #print(src, src.shape)
        #print(dest, dest.shape)
        #print(edge_attr, edge_attr.shape)
        out = torch.cat([src, dest, edge_attr], 1)
        out = self.edge_mlp(out)
        #out = F.relu(out) ### ADDED THIS ###
        #print("Exit edge model")
        return out


class NodeModel(torch.nn.Module):
    def __init__(self, n_node_features, hiddens, n_targets):
        super(NodeModel, self).__init__()
        self.node_mlp_1 = torch.nn.Sequential(
            torch.nn.Linear(n_node_features + hiddens, hiddens),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hiddens, hiddens),
        )
        self.node_mlp_2 = torch.nn.Sequential(
            torch.nn.Linear(n_node_features + hiddens, hiddens),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hiddens, n_targets),
        )

    def forward(self, x, edge_index, edge_attr, u, batch):
        #print("In node model")
        row, col = edge_index
        out = torch.cat([x[col], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter_mean(out, row, dim=0, dim_size=x.size(0))
        out = torch.cat([x, out], dim=1)
        out = self.node_mlp_2(out)
        #print("Exit node model")
        return out

class MetaNet(torch.nn.Module):
    def __init__(self, n_node_features, n_edge_features, num_hidden):
        super(MetaNet, self).__init__()

        # Input Layer
        self.input = MetaLayer(
            edge_model=EdgeModel(
                n_node_features=n_node_features, n_edge_features=n_edge_features,
                hiddens=num_hidden, n_targets=num_hidden),
            node_model=NodeModel(n_node_features=n_node_features, hiddens=num_hidden, n_targets=num_hidden)
            )

        # Output Layer
        self.output = MetaLayer(
            edge_model=EdgeModel(
                n_node_features=num_hidden, n_edge_features=num_hidden,
                hiddens=num_hidden, n_targets=num_hidden),
            node_model=NodeModel(n_node_features=num_hidden, hiddens=num_hidden, n_targets=1)
        )

        # Attention Mechanism
        self.attention = GATv2Conv(in_channels=num_hidden,out_channels=num_hidden,concat=False,dropout=0.1,negative_slope=0.1,heads=2)
    def forward(self, data):
        x, edge_index, edge_attr, y = data.x, data.edge_index, data.edge_attr, data.y
        x, edge_attr, _ = self.input(x, edge_index, edge_attr)
        x = F.relu(x)
        # x = F.dropout(x, p=0.2, training=self.training)

        # Attention Mechanism
        # x = x.unsqueeze(0)
        x = self.attention(x, edge_index)
        x = F.relu(x)
        # x = x.squeeze(0)

        x, edge_attr, _ = self.output(x, edge_index, edge_attr)
        # x = F.relu(x)
        # x = F.dropout(x, p=0.2, training=self.training)
        return x.clamp(min=0)

# MODS TO MODEL END HERE

In [88]:
num_node_features = 21
num_edge_features = 4
num_hidden = 128

In [89]:
model = MetaNet(num_node_features, num_edge_features, num_hidden).to(device)


In [90]:
# model  = Net(num_node_features, num_hidden).to(device)

In [ ]:
model.eval()

In [92]:
optimizer = torch.optim.Adam(lr=1e-3,params=model.parameters())

In [93]:
def train(dataset):
    # Monitor training.
    losses = []

    # Put model in training mode!
    model.train()
    i=0
    for i, batch in enumerate(dataset):
        #print("misaa")
        # Training step.

        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = torch.sqrt(F.mse_loss(out.squeeze(), batch.y.squeeze()))
        #print(f"Training oss for {i}: {loss}")
        loss.backward()
        optimizer.step()
        # Monitoring
        losses.append(loss.item())
        if(i == 1439): break
    # Return training metrics.
    return losses


def evaluate(dataset):
    # Monitor evaluation.
    losses = []

    # Validation (1)
    model.eval()
    i = 0
    for i, batch in enumerate(dataset):
        batch = batch.to(device)

        # Calculate validation losses.
        out = model(batch)
        loss = torch.sqrt(F.mse_loss(out.squeeze(), batch.y.squeeze()))

        # Metric logging.
        losses.append(loss.item())
        if(i == 479): break

    return losses

In [94]:
from sklearn.utils import shuffle
from torch_geometric.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset_valid, batch_size=3, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=True)

In [ ]:
# dataset_sce3=[dataset_test[i] for i in range(80)]
# dataset_sce4=[dataset_test[i] for i in range(80,160)]
# dataset_sce5=[dataset_test[i] for i in range(160,240)]
# dataset_sce6=[dataset_test[i] for i in range(240,320)]
# dataset_sce7=[dataset_test[i] for i in range(320,400)]
# dataset_sce8=[dataset_test[i] for i in range(400,480)]


In [ ]:
for batch in train_loader:
    print(batch)
    break

In [ ]:
# Metrics recorder per epoch.
train_losses = []

valid_losses = []
valid_losses_corrected = []

# Training loop.
model.train()

for epoch in range(2001):
    # Train.
    train_epoch_losses = train(train_loader)
    print(f"Epoch: {epoch}, Len of Training loss: {len(train_epoch_losses)}, Average loss: {float(np.sum(train_epoch_losses))/len(train_epoch_losses)}")
    train_losses.append(np.mean(train_epoch_losses))

    valid_epoch_losses= evaluate(valid_loader)
    print(f"Len of Validation loss: {len(valid_epoch_losses)}, Average loss: {float(np.sum(valid_epoch_losses))/len(valid_epoch_losses)}")
    valid_losses.append(np.mean(valid_epoch_losses))

    if epoch%100==0:
        torch.save(model, f"model_{epoch}.pth")


In [105]:
torch.cuda.empty_cache()

In [97]:
import matplotlib.pyplot as plt

In [ ]:
num_epochs = 2000

plt.plot(np.linspace(1, num_epochs, num_epochs).astype(int), train_losses[:2000], label="Training loss")
plt.plot(np.linspace(1, num_epochs, num_epochs).astype(int), valid_losses[:2000], label="Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# custom code

In [99]:
dataset_sce3=[dataset_test[i] for i in range(80)]
dataset_sce4=[dataset_test[i] for i in range(80,160)]
dataset_sce5=[dataset_test[i] for i in range(160,240)]
dataset_sce6=[dataset_test[i] for i in range(240,320)]
dataset_sce7=[dataset_test[i] for i in range(320,400)]
dataset_sce8=[dataset_test[i] for i in range(400,480)]

In [101]:
sce3_loader = DataLoader(dataset_sce3, batch_size=1, shuffle=True)
sce4_loader = DataLoader(dataset_sce4, batch_size=1, shuffle=True)
sce5_loader = DataLoader(dataset_sce5, batch_size=1, shuffle=True)
sce6_loader = DataLoader(dataset_sce6, batch_size=1, shuffle=True)
sce7_loader = DataLoader(dataset_sce7, batch_size=1, shuffle=True)
sce8_loader = DataLoader(dataset_sce8, batch_size=1, shuffle=True)
test_loader=DataLoader(dataset_test, batch_size=1, shuffle=True)

In [102]:
from sklearn.metrics import r2_score

def find_r2(y_pred, y_true):
     metric = r2_score(y_true=y_true, y_pred=y_pred)
     return metric

In [106]:
mn = 100
prevRmse = 100


for i in range(5, 21):
  modelv2 = torch.load(f"/content/model_{i * 100}.pth")
  # modelv2.to("cpu")
  modelv2.eval()

  def evaluate_test(dataset):
      # Monitor evaluation.
      deployment_rmses=[]
      deployment_maes=[]
      #deployment_r2s=[]
      # y_t_list=[]
      # y_p_list=[]
      # Validation (1)
      modelv2.eval()
      i = 0
      for i, batch in enumerate(dataset):
          batch = batch.to(device)

          # Calculate validation losses.
          out = modelv2(batch)
          deployment_rmse = torch.sqrt(F.mse_loss(out.squeeze(), batch.y.squeeze()))
          deployment_mae=np.mean(np.sqrt(torch.pow(out.view(-1) - batch.y.view(-1), 2).tolist()))
          #deployment_r2= find_r2(out.view(-1), batch.y.view(-1))
          # Metric logging.
          deployment_rmses.append(deployment_rmse.item())
          deployment_maes.append(deployment_mae)
          # y_t_list.extend(out.view(-1).tolist())
          # y_p_list.extend(batch.y.view(-1).tolist())
          #deployment_r2s.append(deployment_r2)
          if(i == 479): break
      avg_rmse=np.mean(deployment_rmses)
      mae=np.mean(deployment_maes)
      r2=0.0 #find_r2(y_t_list,y_p_list)
      return avg_rmse, mae, r2

  full_test_rmse, full_test_mae, full_test_r2= evaluate_test(test_loader)
  if(full_test_rmse < prevRmse):
    prevRmse = full_test_rmse
    mn = i

In [ ]:
print(mn)
print(prevRmse)

In [122]:
torch.cuda.empty_cache()


In [ ]:
modelv2 = torch.load("model_2000.pth")
# modelv2.to("cpu")
modelv2.eval()

In [125]:
def evaluate_test(dataset):
    # Monitor evaluation.
    deployment_rmses=[]
    deployment_maes=[]
    #deployment_r2s=[]
    # y_t_list=[]
    # y_p_list=[]
    # Validation (1)
    modelv2.eval()
    i = 0
    for i, batch in enumerate(dataset):
        batch = batch.to(device)

        # Calculate validation losses.
        out = modelv2(batch)
        deployment_rmse = torch.sqrt(F.mse_loss(out.squeeze(), batch.y.squeeze()))
        deployment_mae=np.mean(np.sqrt(torch.pow(out.view(-1) - batch.y.view(-1), 2).tolist()))
        #deployment_r2= find_r2(out.view(-1), batch.y.view(-1))
        # Metric logging.
        deployment_rmses.append(deployment_rmse.item())
        deployment_maes.append(deployment_mae)
        # y_t_list.extend(out.view(-1).tolist())
        # y_p_list.extend(batch.y.view(-1).tolist())
        #deployment_r2s.append(deployment_r2)
        if(i == 479): break
    avg_rmse=np.mean(deployment_rmses)
    mae=np.mean(deployment_maes)
    r2=0
    return avg_rmse, mae, r2

In [126]:
dataset_sce3=[dataset_test[i] for i in range(80)]
dataset_sce4=[dataset_test[i] for i in range(80,160)]
dataset_sce5=[dataset_test[i] for i in range(160,240)]
dataset_sce6=[dataset_test[i] for i in range(240,320)]
dataset_sce7=[dataset_test[i] for i in range(320,400)]
dataset_sce8=[dataset_test[i] for i in range(400,480)]

In [127]:
sce3_loader = DataLoader(dataset_sce3, batch_size=1, shuffle=True)
sce4_loader = DataLoader(dataset_sce4, batch_size=1, shuffle=True)
sce5_loader = DataLoader(dataset_sce5, batch_size=1, shuffle=True)
sce6_loader = DataLoader(dataset_sce6, batch_size=1, shuffle=True)
sce7_loader = DataLoader(dataset_sce7, batch_size=1, shuffle=True)
sce8_loader = DataLoader(dataset_sce8, batch_size=1, shuffle=True)
test_loader=DataLoader(dataset_test, batch_size=1, shuffle=True)

In [ ]:
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=True)


In [128]:
#evaluate the model on the test set and for separate scenarios
full_test_rmse, full_test_mae, full_test_r2= evaluate_test(test_loader)
# Scenario 3 evaluation
sce3_rmse, sce3_mae, sce3_r2 = evaluate_test(sce3_loader)

# Scenario 4 evaluation
sce4_rmse, sce4_mae, sce4_r2 = evaluate_test(sce4_loader)

# Scenario 5 evaluation
sce5_rmse, sce5_mae, sce5_r2 = evaluate_test(sce5_loader)

# Scenario 6 evaluation
sce6_rmse, sce6_mae, sce6_r2 = evaluate_test(sce6_loader)

# Scenario 7 evaluation
sce7_rmse, sce7_mae, sce7_r2 = evaluate_test(sce7_loader)

# Scenario 8 evaluation
sce8_rmse, sce8_mae, sce8_r2 = evaluate_test(sce8_loader)

In [ ]:
print("Scenario  | Avg. RMSE  | MAE        | R^2")
print("-------------------------------------------------")
print(f"Full Test | {full_test_rmse:.4f} | {full_test_mae:.4f} | {full_test_r2:.4f}")
print(f"Scenario 3| {sce3_rmse:.4f} | {sce3_mae:.4f} | {sce3_r2:.4f}")
print(f"Scenario 4| {sce4_rmse:.4f} | {sce4_mae:.4f} | {sce4_r2:.4f}")
print(f"Scenario 5| {sce5_rmse:.4f} | {sce5_mae:.4f} | {sce5_r2:.4f}")
print(f"Scenario 6| {sce6_rmse:.4f} | {sce6_mae:.4f} | {sce6_r2:.4f}")
print(f"Scenario 7| {sce7_rmse:.4f} | {sce7_mae:.4f} | {sce7_r2:.4f}")
print(f"Scenario 8| {sce8_rmse:.4f} | {sce8_mae:.4f} | {sce8_r2:.4f}")